In [ ]:
import os, json
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

from config import StockformerConfig
from train import train
from inference import init_config, init_model, predict

In [ ]:
load_checkpoint = 'nvda-30d-6-16'
config = init_config(load_checkpoint)
model = init_model(config, load_checkpoint)

In [ ]:
symbols = ['NVDA']
forecasts = predict(symbols, model, config, end_date="2024-07-20")

In [ ]:
for i, forecast in enumerate(forecasts):
    figure, axes = plt.subplots()

    true_pred = yf.download(
        symbols[i],
        start=forecast['Date'].iloc[0].strftime('%Y-%m-%d'),
        end=forecast['Date'].iloc[-1].strftime('%Y-%m-%d'),
    )

    axes.plot(true_pred['Close'].values[:-1], color="red", label="Actual Forecasted Price")
    axes.plot(forecast['Close'], color="green", label="Predicted Forecasted Price")
    axes.plot(forecast['Close'].values[:-config.prediction_length], color="blue", label="Historical Price")
    plt.title(f'{symbols[i]} Forecast')
    plt.xlabel('Trading Days')
    plt.ylabel('Close Price')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
for i, forecast in enumerate(forecasts):
    figure, axes = plt.subplots()

    true_pred = yf.download(
        symbols[i],
        start=forecast['Date'].iloc[0].strftime('%Y-%m-%d'),
        end=forecast['Date'].iloc[-1].strftime('%Y-%m-%d'),
    )

    axes.plot(true_pred['Close'].values[:-1], color="red", label="Actual Forecasted Price")
    axes.plot(forecast['Close'], color="green", label="Predicted Forecasted Price")
    axes.plot(forecast['Close'].values[:-config.prediction_length], color="blue", label="Historical Price")
    plt.title(f'{symbols[i]} Forecast')
    plt.xlabel('Trading Days')
    plt.ylabel('Close Price')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
for i, forecast in enumerate(forecasts):
    figure, axes = plt.subplots()

    true_pred = yf.download(
        symbols[i],
        start=forecast['Date'].iloc[0].strftime('%Y-%m-%d'),
        end=forecast['Date'].iloc[-1].strftime('%Y-%m-%d'),
    )

    axes.plot(true_pred['Close'].values[:-1], color="red", label="Actual Forecasted Price")
    axes.plot(forecast['Close'], color="green", label="Predicted Forecasted Price")
    axes.plot(forecast['Close'].values[:-config.prediction_length], color="blue", label="Historical Price")
    plt.title(f'{symbols[i]} Forecast')
    plt.xlabel('Trading Days')
    plt.ylabel('Close Price')
    plt.legend()
    plt.grid(True)
    plt.show()